# 🧠 SVM-Based VLSI Partitioning Model

This notebook implements a Support Vector Machine (SVM)-based approach for partitioning VLSI netlists represented as graphs.

### Model Logic:
- Each node in the graph represents a circuit component with features:
  - Power, Area, Average Edge Distance, Degree
- Pseudo-labels for supervised learning are generated using **KMeans** clustering.
- An **SVM classifier** is trained on the node features and labels.
- After training, each node is assigned to a partition, and standard partitioning metrics are computed.

### Goals:
- Minimize inter-partition **cut edges**
- Minimize total **wire length**
- Balance **power and area** across partitions
- Reduce **critical path delay**


In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from collections import defaultdict
import random, math


In [ ]:
def extract_node_features(G):
    features = []
    node_ids = []
    for node in G.nodes():
        if node.startswith("IN_") or node.startswith("OUT_"):
            continue
        power = G.nodes[node].get("power", 0)
        area = G.nodes[node].get("area", 0)
        in_edges = G.in_edges(node)
        out_edges = G.out_edges(node)
        edge_distances = [G.edges[u, v].get("distance", 1.0) for u, v in list(in_edges) + list(out_edges)]
        avg_distance = np.mean(edge_distances) if edge_distances else 0.0
        degree = G.in_degree(node) + G.out_degree(node)
        features.append([power, area, avg_distance, degree])
        node_ids.append(node)
    return np.array(features), node_ids


In [ ]:
def train_svm_model(G):
    features, node_ids = extract_node_features(G)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)

    km = KMeans(n_clusters=2, n_init=10)
    y_pseudo = km.fit_predict(X_scaled)

    clf = SVC(kernel='rbf', C=1.0, gamma='scale')
    clf.fit(X_scaled, y_pseudo)
    y_pred = clf.predict(X_scaled)

    return {node_ids[i]: int(y_pred[i]) for i in range(len(node_ids))}


In [ ]:
def compute_partition_metrics(G, partition_map, input_nodes, output_nodes):
    num_cuts = 0
    total_wire_length = 0
    G_weighted = nx.DiGraph()

    for u, v in G.edges():
        wire_length = G.edges[u, v].get("distance", 1)
        cluster_u = partition_map.get(u, -1)
        cluster_v = partition_map.get(v, -1)

        if cluster_u != -1 and cluster_v != -1:
            if cluster_u != cluster_v:
                wire_length *= 10
                num_cuts += 1
        total_wire_length += wire_length
        G_weighted.add_edge(u, v, weight=wire_length)

    while not nx.is_directed_acyclic_graph(G_weighted):
        try:
            cycle = next(nx.simple_cycles(G_weighted))
            min_edge = min(((cycle[i], cycle[(i + 1) % len(cycle)]) for i in range(len(cycle))),
                           key=lambda e: G_weighted.edges[e].get("weight", 1))
            G_weighted.remove_edge(*min_edge)
        except StopIteration:
            break

    critical_delay = 0
    for inp in input_nodes:
        if inp in G_weighted.nodes:
            for out in output_nodes:
                if out in G_weighted.nodes:
                    try:
                        path = nx.dag_longest_path(G_weighted, weight="weight")
                        path_length = sum(G_weighted[u][v]["weight"] for u, v in zip(path, path[1:]))
                        critical_delay = max(critical_delay, path_length)
                    except nx.NetworkXNoPath:
                        continue

    return {
        "cut_edges": num_cuts,
        "total_wire_length": round(total_wire_length, 2),
        "critical_delay": round(critical_delay, 2)
    }


In [ ]:
# Example usage
from utils.graph_gen import generate_netlist

G, inputs, outputs = generate_netlist(num_nodes=50, num_edges=100, seed=42)
partition_map = train_svm_model(G)
metrics = compute_partition_metrics(G, partition_map, inputs, outputs)

print(metrics)
